### Christos Vlassis

In [26]:
# We import the libriaries to use:
import random
import pandas as pd
from itertools import combinations

# 1)  Import and pre-process the dataset with users

In [27]:
# importing the ratings data
Main_ratings = pd.read_csv("C:\\Users\\vlass\\OneDrive\\Υπολογιστής\\Metaptyxiako\\3rd_Trimester\\Mining_Big_Dataset\\Project_1\\Movie Lens Dataset\\Main_ratings.txt", sep="|", header=0)

# importing the users data
Main_users = pd.read_csv("C:\\Users\\vlass\\OneDrive\\Υπολογιστής\\Metaptyxiako\\3rd_Trimester\\Mining_Big_Dataset\\Project_1\\Movie Lens Dataset\\Main_users.txt", sep="|", header=0)

# importing the movies data
Main_movies = pd.read_csv("C:\\Users\\vlass\\OneDrive\\Υπολογιστής\\Metaptyxiako\\3rd_Trimester\\Mining_Big_Dataset\\Project_1\\Movie Lens Dataset\\Main_movies.txt", sep="|", header=0,encoding='ISO-8859-1')

# 2) Compute exact Jaccard similarity of users


In [28]:
# we loop throught the data frame and create a dictionary that will contain the sets.
# each set represents a user_id of  and contains the movies that he has rated.
user_movies = {}
for index, row in Main_ratings.iterrows():
    user_id = row['user_id']
    movie_id = row['movie_id']
    if user_id not in user_movies:
        user_movies[user_id] = set()  # Create a new set if this is a new user_id
    user_movies[user_id].add(movie_id)  # Add the movie_id to the user's set


In [29]:
# we create a function that calculates the jaccard distance:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1) + len(set2) - intersection
    return intersection / union

In [30]:
# We create a list. each list represents a user_id and the sets contains the movie_ids that the users has rated:
i = 1
Main_sets = []
while i <= len(user_movies):  
    Main_sets.append(user_movies[i])
    i += 1
    

#### Note: Because it is a list, user 1 now is called user 0, user 2 is called user 1 e.t.c
#### or, even better, user 1 is in position 0, user 2 in position 1 e.t.c

## Caclulating the Jaccard Similarity

In [31]:
y_true = list()
max_similarity = 0 
for i, j in combinations(range(len(Main_sets)), 2):
    set1 = Main_sets[i]
    set2 = Main_sets[j]
    similarity = jaccard_similarity(set1, set2)
    similarity = round(similarity, 3) 
    if similarity > max_similarity:
        max_similarity = similarity
        max_pairs = (i+1,j+1)
        max_common_movies = set1.intersection(set2)  
    if similarity >= 0.5:
        print(f"The pairs that are similar are {i+1} {j+1}")
        print(f"The Jaccard similarity between these pairs is {similarity}\n")
        # i create the pairs that were found to have the same similarity
        merged_value = str(i) + '-' + str(j)
        y_true.append(merged_value) 


The pairs that are similar are 197 600
The Jaccard similarity between these pairs is 0.5

The pairs that are similar are 197 826
The Jaccard similarity between these pairs is 0.513

The pairs that are similar are 328 788
The Jaccard similarity between these pairs is 0.673

The pairs that are similar are 408 898
The Jaccard similarity between these pairs is 0.839

The pairs that are similar are 451 489
The Jaccard similarity between these pairs is 0.533

The pairs that are similar are 489 587
The Jaccard similarity between these pairs is 0.63

The pairs that are similar are 554 764
The Jaccard similarity between these pairs is 0.517

The pairs that are similar are 600 826
The Jaccard similarity between these pairs is 0.545

The pairs that are similar are 674 879
The Jaccard similarity between these pairs is 0.522

The pairs that are similar are 800 879
The Jaccard similarity between these pairs is 0.5



In [32]:
# lets bring the name of the movies: 
merged_df = pd.merge(Main_movies, Main_ratings, on='movie_id')

# lets keep only the relevant rows: 
selected_rows = merged_df[merged_df['movie_id'].isin(max_common_movies)]

# select the movie titles that are similar for both:
man_common_movie_names = selected_rows['title'].unique()

In [33]:
# We output most similar pair of users, with their common movies and similarity score:
print(f'The most similar pairs are: {max_pairs}')
print(f'With a max similarity of {max_similarity}')
print(f'And their common movies are {man_common_movie_names}')

The most similar pairs are: (408, 898)
With a max similarity of 0.839
And their common movies are ['Kolya' 'Contact' 'Gattaca' 'Starship Troopers' 'Good Will Hunting'
 'English Patient, The' 'Scream' 'Air Force One' 'L.A. Confidential'
 'Rainmaker, The' 'Midnight in the Garden of Good and Evil' 'Titanic'
 'Apt Pupil' 'Everyone Says I Love You' 'Lost Highway' 'Cop Land'
 'Conspiracy Theory' 'U Turn' 'Wag the Dog' 'Spawn' 'Mouse Hunt'
 'Rocket Man' 'Jackal, The' 'Saint, The' 'Tomorrow Never Dies'
 'Indian Summer']


# 3) Compute similarity using Min-hash signatures

### 3.A) Using 50 hash functions for the evaluation of Min-hashing

In [34]:

# hash function
def compute_min_value(a, value, b, R, min_value):
    # Compute new_value
    new_value = (a * value + b) % R
    # Check if new_value is less than min_value
    if new_value < min_value:
        min_value = new_value
    return min_value


The logic for the loops:
We find for every user for all the hash function the min signatures and import them to a new list called list_of_hashes.
This list will have the min-signatures of all the hash functions. We use a seed to change the hash functions but on the same time we have the same hash functions accross all the users.

We append to the list_of_5_dif_functions the lists that contain the min signatures for each run


In [35]:
R = 199999
list_of_5_dif_functions = list()

list_of_seeds = [50,100,250,350,450]
# for 5 different runs
for y in list_of_seeds:
    list_of_hashes = list()
    # for every user
    for sets in Main_sets:
        hash_list = list()
        # for every hash function
        for i in range(1,51): ## change this to 51, 101 OR 201 for more hash functions!
            random.seed(i+y) # call fixed seed. That way we will have the same a and b for all the same hash function accross users
            a  = random.randint(1000, 10990) # create a variable
            b  = random.randint(11000, 20000) # create b variable
            min_value = 200000
            # for every movie of the user 
            # calculate the min signature 
            for value in sets:
                min_value = compute_min_value(a, value, b, R, min_value)
            hash_list.append(min_value)
        list_of_hashes.append(hash_list)  
    list_of_5_dif_functions.append(list_of_hashes)
    
    
    

Now that we have the lists with the min signatures for all the users we find the similarity with the use of the min signatures. At the same time we find the pairs that are similar according to the min signature method. This is information is contained in the y_pred list. We will only output the users with similarity greater than 0.5.

In [36]:
# calculate the y_pred. This will be used to calculate the False positive and False negative.
# We do this for all functions.
y_pred = list()
y_pred_Second_function = list() 
y_pred_Third_function = list() 
y_pred_Fourth_function = list() 
y_pred_Fifth_function = list() 
for indexer in range(0,5):
    # i and j moves throught the lists. For all the combinations of lists!
    # values moves the values of the lists
    for i, j in combinations(range(len(list_of_5_dif_functions[indexer])), 2): 
        counter = 0 
        total_counter = 0 
        ratio = 0
        # when this loop ends, i will have passed thought the first combination of lists:
        for values in range(len(list_of_5_dif_functions[indexer][i])): 
            total_counter = total_counter + 1
            if list_of_5_dif_functions[indexer][i][values] == list_of_5_dif_functions[indexer][j][values]: 
                counter = counter + 1
        ratio = counter/total_counter
        ratio = round(ratio,2)
        if ratio>= 0.5:
            # i create the pairs that were found to have the same similarity
            ## indexer enumerates throught the different lists. Each of the list is calculated with a different hash function
            if indexer == 0:
                merged_value = str(i) + '-' + str(j)
                y_pred.append(merged_value)
                print(f'For the 1st run the users {i+1} and {j+1} have similarity equal to: {ratio}')
            elif indexer == 1: 
                merged_value_Second_function = str(i) + '-' + str(j) 
                y_pred_Second_function.append(merged_value_Second_function) 
            elif indexer == 2:
                merged_value_Third_function = str(i) + '-' + str(j) 
                y_pred_Third_function.append(merged_value_Third_function) 
            elif indexer == 3:
                merged_value_Fourth_function = str(i) + '-' + str(j) 
                y_pred_Fourth_function.append(merged_value_Fourth_function) 
            else:
                merged_value_Fifth_function = str(i) + '-' + str(j) 
                y_pred_Fifth_function.append(merged_value_Fifth_function) 

For the 1st run the users 7 and 194 have similarity equal to: 0.5
For the 1st run the users 7 and 234 have similarity equal to: 0.54
For the 1st run the users 7 and 311 have similarity equal to: 0.5
For the 1st run the users 7 and 326 have similarity equal to: 0.52
For the 1st run the users 7 and 650 have similarity equal to: 0.56
For the 1st run the users 7 and 788 have similarity equal to: 0.5
For the 1st run the users 7 and 846 have similarity equal to: 0.5
For the 1st run the users 33 and 616 have similarity equal to: 0.5
For the 1st run the users 35 and 362 have similarity equal to: 0.5
For the 1st run the users 37 and 217 have similarity equal to: 0.54
For the 1st run the users 40 and 915 have similarity equal to: 0.56
For the 1st run the users 42 and 280 have similarity equal to: 0.5
For the 1st run the users 42 and 378 have similarity equal to: 0.5
For the 1st run the users 59 and 429 have similarity equal to: 0.5
For the 1st run the users 59 and 533 have similarity equal to: 0

For the 1st run the users 446 and 853 have similarity equal to: 0.5
For the 1st run the users 450 and 474 have similarity equal to: 0.52
For the 1st run the users 450 and 524 have similarity equal to: 0.5
For the 1st run the users 450 and 846 have similarity equal to: 0.52
For the 1st run the users 451 and 489 have similarity equal to: 0.5
For the 1st run the users 451 and 587 have similarity equal to: 0.5
For the 1st run the users 457 and 586 have similarity equal to: 0.54
For the 1st run the users 457 and 846 have similarity equal to: 0.5
For the 1st run the users 457 and 864 have similarity equal to: 0.58
For the 1st run the users 457 and 916 have similarity equal to: 0.5
For the 1st run the users 457 and 943 have similarity equal to: 0.5
For the 1st run the users 484 and 586 have similarity equal to: 0.52
For the 1st run the users 485 and 547 have similarity equal to: 0.52
For the 1st run the users 489 and 587 have similarity equal to: 0.64
For the 1st run the users 489 and 724 hav

Now its time to calculate the False Positives and False Negatives. We have 2 lists in our disposal. The y_true which contains the pairs of users that were found similar according to the Jacard similarity (>=0.5). We also have the y_pred which contains the pairs of users that were found similar according to the min signature method (>=0.5).

The logic: We find the True positives and from there we find the false positives and false negatives:

True positives: are the values between the 2 lists that are equal. The pair exists in both Lists.
False positives: The pair will be found in Hash but not in Jaccard. 
False negatives: The pair will not  be found in Hash but will be found in Jaccard.

The following code find the True positives and later we compute the false positives and false negatives.

In [37]:
true_positives = 0
true_positives_Second_f = 0
true_positives_Third_f = 0
true_positives_Fourth_f = 0
true_positives_Fifth_f = 0

positive_predictions = len(y_pred)
positve_actual = len(y_true)
# for the values of the 'true' list
for pairs_true in range(len(y_true)):
    # for the values of the 'pred' list
    for pairs_pred in range(len(y_pred)):
        # Find the True Positives:
        if y_true[pairs_true] == y_pred[pairs_pred]:
            true_positives = true_positives + 1
    for pairs_pred in range(len(y_pred_Second_function)): 
        # Find the True Positives:
        if y_true[pairs_true] == y_pred_Second_function[pairs_pred]:
            true_positives_Second_f = true_positives_Second_f + 1         
    for pairs_pred in range(len(y_pred_Third_function)): 
        # Find the True Positives:
        if y_true[pairs_true] == y_pred_Third_function[pairs_pred]:
            true_positives_Third_f = true_positives_Third_f + 1 
    for pairs_pred in range(len(y_pred_Fourth_function)): 
        # Find the True Positives:
        if y_true[pairs_true] == y_pred_Fourth_function[pairs_pred]:
            true_positives_Fourth_f = true_positives_Fourth_f + 1 
    for pairs_pred in range(len(y_pred_Fifth_function)): 
        # Find the True Positives:
        if y_true[pairs_true] == y_pred_Fifth_function[pairs_pred]:
            true_positives_Fifth_f = true_positives_Fifth_f + 1 
    
    
    
               
# calculating the False Positives:
false_positives = positive_predictions - true_positives 
false_positives_Second_Function = positive_predictions - true_positives_Second_f
false_positives_Third_Function = positive_predictions - true_positives_Third_f
false_positives_Fourth_Function = positive_predictions - true_positives_Fourth_f
false_positives_Fifth_Function = positive_predictions - true_positives_Fifth_f

false_positives_avg = (false_positives+false_positives_Second_Function+false_positives_Third_Function+false_positives_Fourth_Function+false_positives_Fifth_Function)/5

# calculating the False Negatives:
false_negatives = positve_actual - true_positives 
false_negatives_Second_Function = positve_actual - true_positives_Second_f 
false_negatives_Third_Function = positve_actual - true_positives_Third_f 
false_negatives_Fourth_Function = positve_actual - true_positives_Fourth_f 
false_negatives_Fifth_Function = positve_actual - true_positives_Fifth_f 

false_negatives_avg = (false_negatives+false_negatives_Second_Function+false_negatives_Third_Function+false_negatives_Fourth_Function+false_negatives_Fifth_Function)/5

print(f'For the 1st run we found {false_positives} false positives and {false_negatives} false negatives')
print(f'For 5 runs we found an average of {false_positives_avg} false positives and {false_negatives_avg} false negatives')



For the 1st run we found 168 false positives and 0 false negatives
For 5 runs we found an average of 169.2 false positives and 1.2 false negatives


We will do the same thing for 100 and 200 hash functions. I will not comment on these since we use exactly the same code.

### 3.B) Using 100 hash functions for the evaluation of Min-hashing

In [38]:
R = 199999
# all the following 'list_of_hashes...' contain for each function the min signatures
list_of_5_dif_functions = list()

list_of_seeds = [50,100,250,350,450]
# for 5 different runs
for y in list_of_seeds:
    list_of_hashes = list()
    # for every user
    for sets in Main_sets:
        hash_list = list()
        # for every hash function
        for i in range(1,101): ## change this to 51, 101 OR 201 for more hash functions!
            random.seed(i+y) # call fixed seed. That way we will have the same a and b for all the same hash function accross users
            a  = random.randint(1000, 10990) # create a variable
            b  = random.randint(11000, 20000) # create b variable
            min_value = 200000
            # for every movie of the user 
            # calculate the min signature 
            for value in sets:
                min_value = compute_min_value(a, value, b, R, min_value)
            hash_list.append(min_value)
        list_of_hashes.append(hash_list)  
    list_of_5_dif_functions.append(list_of_hashes)
    
    
    

In [39]:
# calculate the y_pred. This will be used to calculate the False positive and False negative.
# We do this for all functions.
y_pred = list()
y_pred_Second_function = list() 
y_pred_Third_function = list() 
y_pred_Fourth_function = list() 
y_pred_Fifth_function = list() 
for indexer in range(0,5):
    # i and j moves throught the lists. For all the combinations of lists!
    # values moves the values of the lists
    for i, j in combinations(range(len(list_of_5_dif_functions[indexer])), 2): 
        counter = 0 
        total_counter = 0 
        ratio = 0
        # when this loop ends, i will have passed thought the first combination of lists:
        for values in range(len(list_of_5_dif_functions[indexer][i])): 
            total_counter = total_counter + 1
            if list_of_5_dif_functions[indexer][i][values] == list_of_5_dif_functions[indexer][j][values]: 
                counter = counter + 1
        ratio = counter/total_counter
        ratio = round(ratio,2)
        if ratio>= 0.5:
            # i create the pairs that were found to have the same similarity
            ## indexer enumerates throught the different lists. Each of the list is calculated with a different hash function
            if indexer == 0:
                merged_value = str(i) + '-' + str(j)
                y_pred.append(merged_value)
                print(f'For the 1st run the users {i+1} and {j+1} have similarity equal to: {ratio}')
            elif indexer == 1: 
                merged_value_Second_function = str(i) + '-' + str(j) 
                y_pred_Second_function.append(merged_value_Second_function) 
            elif indexer == 2:
                merged_value_Third_function = str(i) + '-' + str(j) 
                y_pred_Third_function.append(merged_value_Third_function) 
            elif indexer == 3:
                merged_value_Fourth_function = str(i) + '-' + str(j) 
                y_pred_Fourth_function.append(merged_value_Fourth_function) 
            else:
                merged_value_Fifth_function = str(i) + '-' + str(j) 
                y_pred_Fifth_function.append(merged_value_Fifth_function) 

For the 1st run the users 7 and 650 have similarity equal to: 0.51
For the 1st run the users 18 and 234 have similarity equal to: 0.5
For the 1st run the users 59 and 308 have similarity equal to: 0.5
For the 1st run the users 92 and 276 have similarity equal to: 0.55
For the 1st run the users 92 and 457 have similarity equal to: 0.5
For the 1st run the users 94 and 682 have similarity equal to: 0.53
For the 1st run the users 94 and 916 have similarity equal to: 0.52
For the 1st run the users 112 and 772 have similarity equal to: 0.57
For the 1st run the users 130 and 276 have similarity equal to: 0.54
For the 1st run the users 133 and 166 have similarity equal to: 0.52
For the 1st run the users 140 and 155 have similarity equal to: 0.53
For the 1st run the users 151 and 474 have similarity equal to: 0.51
For the 1st run the users 194 and 892 have similarity equal to: 0.5
For the 1st run the users 197 and 600 have similarity equal to: 0.59
For the 1st run the users 197 and 826 have sim

In [40]:
true_positives = 0
true_positives_Second_f = 0
true_positives_Third_f = 0
true_positives_Fourth_f = 0
true_positives_Fifth_f = 0

positive_predictions = len(y_pred)
positve_actual = len(y_true)
# for the values of the 'true' list
for pairs_true in range(len(y_true)):
    # for the values of the 'pred' list
    for pairs_pred in range(len(y_pred)):
        # Find the True Positives:
        if y_true[pairs_true] == y_pred[pairs_pred]:
            true_positives = true_positives + 1
    for pairs_pred in range(len(y_pred_Second_function)): 
        # Find the True Positives:
        if y_true[pairs_true] == y_pred_Second_function[pairs_pred]:
            true_positives_Second_f = true_positives_Second_f + 1         
    for pairs_pred in range(len(y_pred_Third_function)): 
        # Find the True Positives:
        if y_true[pairs_true] == y_pred_Third_function[pairs_pred]:
            true_positives_Third_f = true_positives_Third_f + 1 
    for pairs_pred in range(len(y_pred_Fourth_function)): 
        # Find the True Positives:
        if y_true[pairs_true] == y_pred_Fourth_function[pairs_pred]:
            true_positives_Fourth_f = true_positives_Fourth_f + 1 
    for pairs_pred in range(len(y_pred_Fifth_function)): 
        # Find the True Positives:
        if y_true[pairs_true] == y_pred_Fifth_function[pairs_pred]:
            true_positives_Fifth_f = true_positives_Fifth_f + 1 
    
    
    
               
# calculating the False Positives:
false_positives = positive_predictions - true_positives 
false_positives_Second_Function = positive_predictions - true_positives_Second_f
false_positives_Third_Function = positive_predictions - true_positives_Third_f
false_positives_Fourth_Function = positive_predictions - true_positives_Fourth_f
false_positives_Fifth_Function = positive_predictions - true_positives_Fifth_f

false_positives_avg = (false_positives+false_positives_Second_Function+false_positives_Third_Function+false_positives_Fourth_Function+false_positives_Fifth_Function)/5

# calculating the False Negatives:
false_negatives = positve_actual - true_positives 
false_negatives_Second_Function = positve_actual - true_positives_Second_f 
false_negatives_Third_Function = positve_actual - true_positives_Third_f 
false_negatives_Fourth_Function = positve_actual - true_positives_Fourth_f 
false_negatives_Fifth_Function = positve_actual - true_positives_Fifth_f 

false_negatives_avg = (false_negatives+false_negatives_Second_Function+false_negatives_Third_Function+false_negatives_Fourth_Function+false_negatives_Fifth_Function)/5

print(f'For the 1st run we found {false_positives} false positives and {false_negatives} false negatives')
print(f'For 5 runs we found an average of {false_positives_avg} false positives and {false_negatives_avg} false negatives')



For the 1st run we found 43 false positives and 1 false negatives
For 5 runs we found an average of 44.2 false positives and 2.2 false negatives


### 3.C) Using 200 hash functions for the evaluation of Min-hashing

In [41]:
R = 199999
# all the following 'list_of_hashes...' contain for each function the min signatures
list_of_5_dif_functions = list()

list_of_seeds = [50,100,250,350,450]
# for 5 different runs
for y in list_of_seeds:
    list_of_hashes = list()
    # for every user
    for sets in Main_sets:
        hash_list = list()
        # for every hash function
        for i in range(1,201): ## change this to 51, 101 OR 201 for more hash functions!
            random.seed(i+y) # call fixed seed. That way we will have the same a and b for all the same hash function accross users
            a  = random.randint(1000, 10990) # create a variable
            b  = random.randint(11000, 20000) # create b variable
            min_value = 200000
            # for every movie of the user 
            # calculate the min signature 
            for value in sets:
                min_value = compute_min_value(a, value, b, R, min_value)
            hash_list.append(min_value)
        list_of_hashes.append(hash_list)  
    list_of_5_dif_functions.append(list_of_hashes)
    
    
    

In [42]:
# calculate the y_pred. This will be used to calculate the False positive and False negative.
# We do this for all functions.
y_pred = list()
y_pred_Second_function = list() 
y_pred_Third_function = list() 
y_pred_Fourth_function = list() 
y_pred_Fifth_function = list() 
for indexer in range(0,5):
    # i and j moves throught the lists. For all the combinations of lists!
    # values moves the values of the lists
    for i, j in combinations(range(len(list_of_5_dif_functions[indexer])), 2): 
        counter = 0 
        total_counter = 0 
        ratio = 0
        # when this loop ends, i will have passed thought the first combination of lists:
        for values in range(len(list_of_5_dif_functions[indexer][i])): 
            total_counter = total_counter + 1
            if list_of_5_dif_functions[indexer][i][values] == list_of_5_dif_functions[indexer][j][values]: 
                counter = counter + 1
        ratio = counter/total_counter
        ratio = round(ratio,2)
        if ratio>= 0.5:
            # i create the pairs that were found to have the same similarity
            ## indexer enumerates throught the different lists. Each of the list is calculated with a different hash function
            if indexer == 0:
                merged_value = str(i) + '-' + str(j)
                y_pred.append(merged_value)
                print(f'For the 1st run the users {i+1} and {j+1} have similarity equal to: {ratio}')
            elif indexer == 1: 
                merged_value_Second_function = str(i) + '-' + str(j) 
                y_pred_Second_function.append(merged_value_Second_function) 
            elif indexer == 2:
                merged_value_Third_function = str(i) + '-' + str(j) 
                y_pred_Third_function.append(merged_value_Third_function) 
            elif indexer == 3:
                merged_value_Fourth_function = str(i) + '-' + str(j) 
                y_pred_Fourth_function.append(merged_value_Fourth_function) 
            else:
                merged_value_Fifth_function = str(i) + '-' + str(j) 
                y_pred_Fifth_function.append(merged_value_Fifth_function) 

For the 1st run the users 7 and 650 have similarity equal to: 0.52
For the 1st run the users 112 and 772 have similarity equal to: 0.52
For the 1st run the users 140 and 155 have similarity equal to: 0.54
For the 1st run the users 151 and 716 have similarity equal to: 0.5
For the 1st run the users 197 and 600 have similarity equal to: 0.54
For the 1st run the users 197 and 826 have similarity equal to: 0.56
For the 1st run the users 276 and 303 have similarity equal to: 0.51
For the 1st run the users 328 and 788 have similarity equal to: 0.67
For the 1st run the users 356 and 856 have similarity equal to: 0.62
For the 1st run the users 362 and 772 have similarity equal to: 0.52
For the 1st run the users 408 and 898 have similarity equal to: 0.81
For the 1st run the users 416 and 450 have similarity equal to: 0.51
For the 1st run the users 446 and 853 have similarity equal to: 0.51
For the 1st run the users 451 and 489 have similarity equal to: 0.54
For the 1st run the users 489 and 587

In [43]:
true_positives = 0
true_positives_Second_f = 0
true_positives_Third_f = 0
true_positives_Fourth_f = 0
true_positives_Fifth_f = 0

positive_predictions = len(y_pred)
positve_actual = len(y_true)
# for the values of the 'true' list
for pairs_true in range(len(y_true)):
    # for the values of the 'pred' list
    for pairs_pred in range(len(y_pred)):
        # Find the True Positives:
        if y_true[pairs_true] == y_pred[pairs_pred]:
            true_positives = true_positives + 1
    for pairs_pred in range(len(y_pred_Second_function)): 
        # Find the True Positives:
        if y_true[pairs_true] == y_pred_Second_function[pairs_pred]:
            true_positives_Second_f = true_positives_Second_f + 1         
    for pairs_pred in range(len(y_pred_Third_function)): 
        # Find the True Positives:
        if y_true[pairs_true] == y_pred_Third_function[pairs_pred]:
            true_positives_Third_f = true_positives_Third_f + 1 
    for pairs_pred in range(len(y_pred_Fourth_function)): 
        # Find the True Positives:
        if y_true[pairs_true] == y_pred_Fourth_function[pairs_pred]:
            true_positives_Fourth_f = true_positives_Fourth_f + 1 
    for pairs_pred in range(len(y_pred_Fifth_function)): 
        # Find the True Positives:
        if y_true[pairs_true] == y_pred_Fifth_function[pairs_pred]:
            true_positives_Fifth_f = true_positives_Fifth_f + 1 
    
    
    
               
# calculating the False Positives:
false_positives = positive_predictions - true_positives 
false_positives_Second_Function = positive_predictions - true_positives_Second_f
false_positives_Third_Function = positive_predictions - true_positives_Third_f
false_positives_Fourth_Function = positive_predictions - true_positives_Fourth_f
false_positives_Fifth_Function = positive_predictions - true_positives_Fifth_f

false_positives_avg = (false_positives+false_positives_Second_Function+false_positives_Third_Function+false_positives_Fourth_Function+false_positives_Fifth_Function)/5

# calculating the False Negatives:
false_negatives = positve_actual - true_positives 
false_negatives_Second_Function = positve_actual - true_positives_Second_f 
false_negatives_Third_Function = positve_actual - true_positives_Third_f 
false_negatives_Fourth_Function = positve_actual - true_positives_Fourth_f 
false_negatives_Fifth_Function = positve_actual - true_positives_Fifth_f 

false_negatives_avg = (false_negatives+false_negatives_Second_Function+false_negatives_Third_Function+false_negatives_Fourth_Function+false_negatives_Fifth_Function)/5

print(f'For the 1st run we found {false_positives} false positives and {false_negatives} false negatives')
print(f'For 5 runs we found an average of {false_positives_avg} false positives and {false_negatives_avg} false negatives')



For the 1st run we found 13 false positives and 1 false negatives
For 5 runs we found an average of 13.6 false positives and 1.6 false negatives


As we can see the more hash function we use the less are the false positives and false negatives. So, if we have many hash function and compute many min signatures we have more accurate results but we need more computing power and more time.

# 4) Locate similar users using LSH index


Firstly i create a function to split the list into bands as LSH commands. 

In [44]:
# this function is used to split to bands and rows:
def split_to_bands(signature, b):
    assert len(signature) % b == 0
    r = int(len(signature) / b)
    
    subvecs = []
    for i in range(0,len(signature), r):
        subvecs.append(signature[i : i+r])
    return subvecs


In [45]:
# add to a list the lists of min signatures for the 5 different functions:
lists_of_lists_of_hashes = list_of_5_dif_functions

In [46]:
# we will store to a list of lists of lists the splited signatures of each user.
# the big list contains the users with their splited signatures.
# The big_big_list_... contains for the 5 different functions(runs) the splited min signatures for each user
big_big_list_25 = list()
big_big_list_40 = list()
for lists in range(len(lists_of_lists_of_hashes)):
    big_list_25 = list()
    big_list_40 = list()
    for list_of_lists in range(len(lists_of_lists_of_hashes[lists])):
        splited_25 = split_to_bands(lists_of_lists_of_hashes[lists][list_of_lists], 25)
        splited_40 = split_to_bands(lists_of_lists_of_hashes[lists][list_of_lists], 40)
        big_list_25.append(splited_25)
        big_list_40.append(splited_40)
        
    big_big_list_25.append(big_list_25)
    big_big_list_40.append(big_list_40)

In [47]:
# The LSH_sim_per_function_25 contains for the 5 different functions for 25 bands the similar candidates
LSH_sim_per_function_25 = list()
for lists in range(len(big_big_list_25)):
    # we find the candidate similar pairs according to LSH method:
    candidate_set_25 = set()
    # for 25 bands:
    # for combination of users:
    for i, j in combinations(range(len(big_big_list_25[lists])), 2):
        # for the bands of the combination of users:
        for bands in range(len(big_big_list_25[lists][i])):
            # if bands are equal then add users to the candidate set 
            if big_big_list_25[lists][i][bands] == big_big_list_25[lists][j][bands]:
                candidate_set_25.add(str(i) + '-' + str(j))
                break
    LSH_sim_per_function_25.append(candidate_set_25)

In [48]:
# The LSH_sim_per_function_40 contains for the 5 different functions for 40 bands the similar candidates
LSH_sim_per_function_40 = list()
for lists in range(len(big_big_list_40)):
    # we find the candidate similar pairs according to LSH method:
    candidate_set_40 = set()
    # for 40 bands:
    # for combination of users:
    for i, j in combinations(range(len(big_big_list_40[lists])), 2):
        # for the bands of the combination of users:
        for bands in range(len(big_big_list_40[lists][i])):
            # if bands are equal then add users to the candidate set 
            if big_big_list_40[lists][i][bands] == big_big_list_40[lists][j][bands]:
                candidate_set_40.add(str(i) + '-' + str(j))
                break 
    LSH_sim_per_function_40.append(candidate_set_40)

In [49]:
# this loops checks if the jaccard method and LSH method have found the same similar users.
# for 25 bands:
similars_per_function_25 = list()
for functions in range(len(LSH_sim_per_function_25)):
    count_25 = 0    
    for values in range(len(y_true)):
        for items in LSH_sim_per_function_25[functions]:
            if items == y_true[values]:
                count_25 = count_25 + 1    
    similars_per_function_25.append(count_25)
    
print(f'with the use of the first function for 200 hash functions and 25 bands we found {similars_per_function_25[0]} true positives')
print(f'with the use of the second function for 200 hash functions and 25 bands we found {similars_per_function_25[1]} true positives')
print(f'with the use of the third function for 200 hash functions and 25 bands we found {similars_per_function_25[2]} true positives')
print(f'with the use of the fourth function for 200 hash functions and 25 bands we found {similars_per_function_25[3]} true positives')
print(f'with the use of the fifth function for 200 hash functions and 25 bands we found {similars_per_function_25[4]} true positives')

average_true_positives_25 = sum(similars_per_function_25) / 5
print(f'the average true positives for 5 runs for 25 bands is {average_true_positives_25}')

with the use of the first function for 200 hash functions and 25 bands we found 3 true positives
with the use of the second function for 200 hash functions and 25 bands we found 2 true positives
with the use of the third function for 200 hash functions and 25 bands we found 4 true positives
with the use of the fourth function for 200 hash functions and 25 bands we found 4 true positives
with the use of the fifth function for 200 hash functions and 25 bands we found 3 true positives
the average true positives for 5 runs for 25 bands is 3.2


In [50]:
# for 40 bands:
similars_per_function_40 = list()
for functions in range(len(LSH_sim_per_function_40)):
    count_40 = 0
    for values in range(len(y_true)):
        for items in LSH_sim_per_function_40[functions]:
            if items == y_true[values]:
                count_40 = count_40 + 1
    similars_per_function_40.append(count_40)

print(f'with the use of the first function for 200 hash functions and 40 bands we found {similars_per_function_40[0]} true positives')
print(f'with the use of the second function for 200 hash functions and 40 bands we found {similars_per_function_40[1]} true positives')
print(f'with the use of the third function for 200 hash functions and 40 bands we found {similars_per_function_40[2]} true positives')
print(f'with the use of the fourth function for 200 hash functions and 40 bands we found {similars_per_function_40[3]} true positives')
print(f'with the use of the fifth function for 200 hash functions and 40 bands we found {similars_per_function_40[4]} true positives')

average_true_positives_40 = sum(similars_per_function_40) / 5
print(f'the average true positives for 5 runs for 40 bands is {average_true_positives_40}')


with the use of the first function for 200 hash functions and 40 bands we found 7 true positives
with the use of the second function for 200 hash functions and 40 bands we found 6 true positives
with the use of the third function for 200 hash functions and 40 bands we found 6 true positives
with the use of the fourth function for 200 hash functions and 40 bands we found 8 true positives
with the use of the fifth function for 200 hash functions and 40 bands we found 10 true positives
the average true positives for 5 runs for 40 bands is 7.4


As we can see the larger the number of bands the better the results.

The number of evaluations performed are as many as the combination of users that were found to be candidate pairs by the LSH. This is the case, becasuse we compare the candidate pairs with the pairs that we got with the Jaccard method. With that being said for the first hash function there are 7 evaluations made.

With the use of the Jaccard distance we have to compare 943*942/2 = 444,153 different pairs of users and find their similarity.
One the other hand with the use of LSH we can make less comparisons since, if we find a band of a user to be equal to the same band of another user we can break the loop and make less comparisons. So, LSH is faster and needs less computing power to run because we can avoid comparing every pair.

Now, if we use 25 bands we find, an average of 3 true positives. We also know that according to jaccard similarity with a threshold of 0.5 we found 10 users to be similar. So, with the use of the LSH we are losing some pairs that according to Jaccard are similar. Maybe, if we tune the LSH we will find better results.